In [1]:
#import/instalations
!pip install yfinance
import yfinance as yf
import pandas as pd
import requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.0 MB/s eta 0:00:00


# Forex data

In [2]:
#FUNCTIONS

#Download forex data for 1 pair from last 7 days in 1 minute intervals 
def download_data(cur_1, cur_2, t_period='7d', t_interval='1m'):
    #DOWNLOAD DATA
    data = yf.Ticker(f'{cur_1}{cur_2}=X')
    f_data = data.history(period=t_period, interval=t_interval)
    #REMOVE TABLES WITH 0
    mask = f_data.ne(0).any(axis=0)
    f_data = f_data.loc[:, mask]
    #RENAME COLUMNS
    f_data.rename(columns={'Open':f'{cur_1}{cur_2}_OPEN', 'High':f'{cur_1}{cur_2}_HIGH', 
                         'Low':f'{cur_1}{cur_2}_LOW', 'Close':f'{cur_1}{cur_2}_CLOSE'}, inplace=True)
    return f_data

#Download new forex data and merge it with old one
def update_forex_data(old_data_path):
    #LOAD OLD DATA GMT +1
    old_data = pd.read_excel(old_data_path)
    old_data.set_index('Datetime', inplace=True)

    #CURRENCIES PAIRS WHICH WE WANT DOWNLOAD
    main_cur = ['PLN', 'EUR']
    additional_cur = ['CZK', 'HUF', 'USD', 'CHF', 'GBP', 'JPY']

    #DOWNLOAD OUR MAIN CURRENCY PAIR
    forex_data = download_data('EUR', 'PLN')

    #DOWNLOAD OUR ADDITIONAL CURRENCY PAIRS
    for main in main_cur:
        for add in additional_cur:
            #DOWNLOAD PAIR DATA
            temp_data = download_data(main, add)
            #JOIN TO MAIN TABLE
            forex_data = forex_data.join(temp_data)

    #CHANGE DATETIME TO STRING IN ORDER TO SAVE TO XLSX
    forex_data.sort_index(ascending=False, inplace=True)
    forex_data.index = forex_data.index.strftime('%d-%m-%Y %H:%M:%S %z')

    #JOIN OLD AND NEW DATA(FROM THIS WEEK)
    forex_data = pd.concat([forex_data, old_data])

    #REMOVE DUPLICATES
    forex_data.reset_index(inplace=True)
    forex_data.drop_duplicates(['Datetime'], inplace=True)
    forex_data.set_index('Datetime', inplace=True)

    #SAVE TO EXCEL
    forex_data.to_excel('FOREX_DATA.xlsx')
    
#Download daily forex data - DEPRECIATED use "download_data"
def daily_forex_data(pair="EUR/PLN", interval="1day", size="5000"):
    url = "https://twelve-data1.p.rapidapi.com/time_series"
    querystring = {"symbol":pair,"interval":interval,"outputsize":size,"format":"json"}

    headers = {
        "X-RapidAPI-Key": "863b6e82d7msha3b96a4e153c426p11a206jsn073ad98d5070",
        "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring).json()
    return pd.DataFrame(response['values'])    

In [3]:
#Update forex data
forex_path = '/kaggle/input/forex-data-gatherer/FOREX_DATA.xlsx'
update_forex_data(forex_path)

In [ ]:
#Daily forex data
daily_forex = download_data('EUR', 'PLN', '5000d', '1d')

# Economic calendar data

In [ ]:
#Create dates pairs for economic calendar(calendar max offset is 30 days)
dates_pairs = []
initial_date = "2010-01-01"
end_date_init = pd.Timestamp.today().normalize()
start_date = pd.to_datetime(initial_date)

while start_date <= end_date_init:
    end_date = (start_date + pd.offsets.Day(30))
    dates_pairs.append((start_date.date().isoformat(), end_date.date().isoformat()))
    start_date = end_date

In [ ]:
#Create economic calendar with all possible fetched data
economic_data = pd.DataFrame()
for pair in dates_pairs:
    try:
        url = 'https://economic-calendar.tradingview.com/events'
        payload = {
            'from': pair[0],
            'to': pair[1]
        }
        data = requests.get(url, params=payload).json()
        calendar_df = pd.DataFrame(data['result'])

        #Drop unimportant columns and rows
        calendar_drop = ['id', 'period', 'source', 'currency', 'ticker', 'unit', 'scale']
        calendar_df.drop(calendar_drop, axis=1, inplace=True)
        calendar_df.dropna(subset=['actual'], inplace=True)

        #Sort and convert time to GMT +1
        calendar_df = calendar_df.sort_values(by='date', ascending=False)
        calendar_df['date'] = pd.to_datetime(calendar_df['date'], dayfirst=True).dt.tz_convert('Europe/London')
        calendar_df['date'] = calendar_df.date.dt.strftime('%d-%m-%Y %H:%M:%S %z')

        calendar_df.set_index('date', inplace=True)
        economic_data = pd.concat([economic_data, calendar_df])
    except:
        pass
economic_data.to_excel("economic_data.xlsx")    

# Read data

In [2]:
#Economic calendar d.aggregateta
economic_df = pd.read_excel('/kaggle/input/economic-calendar-data/economic_data.xlsx')

In [204]:
economic_poland

,date,title,country,indicator,comment,actual,previous,forecast,importance
85,14-03-2013 13:00:00 +0000,Current Account,PL,Current Account,Current Account is the sum of the balance of t...,-1.55,-1.20,-1.58,-1
86,14-03-2013 13:00:00 +0000,Inflation Rate YoY,PL,Inflation Rate,"In Poland, the most important categories in th...",1.30,1.70,1.50,-1
138,13-03-2013 13:00:00 +0000,Balance of Trade,PL,Balance of Trade,"From the beginning of the past decade, Poland’...",-233.00,-936.00,NaN,-1
365,07-03-2013 13:00:00 +0000,Central Bank FX Reserves,PL,Calendar,NaN,107.10,108.00,NaN,-1
428,06-03-2013 12:00:00 +0000,Interest Rate Decision,PL,Interest Rate,In Poland interest rates decisions are taken b...,3.25,3.75,3.50,0
...,...,...,...,...,...,...,...,...,...
174947,26-04-2023 09:00:00 +0100,Unemployment Rate,PL,Unemployment Rate,"In Poland, the unemployment rate measures the ...",5.40,5.50,5.40,0
175005,25-04-2023 13:00:00 +0100,M3 Money Supply YoY,PL,Money Supply M3,Poland Money Supply M3 includes M2 plus long-t...,6.50,7.40,7.20,-1
175074,24-04-2023 09:00:00 +0100,Industrial Production YoY,PL,Industrial Production,"In Poland, industrial production measures the ...",-2.90,-1.00,-1.90,-1
175075,24-04-2023 09:00:00 +0100,PPI YoY,PL,Producer Prices Change,Producer prices change refers to year over yea...,10.10,18.20,11.00,-1


In [203]:
economic_poland.loc[85]['actual']=10


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [193]:
economic_poland.loc[85]['date']

'14-03-2013 13:00:00 +0000'

In [175]:
economic_poland = economic_df[economic_df['country']=='PL']

mask = economic_poland.duplicated(subset=['date', 'title'], keep=False)
for idx, [index, row] in enumerate(economic_poland.loc[mask].iterrows()):
    try:
        pair = economic_poland.loc[mask].iloc[idx+1]
        if all(row[['date', 'title']] == pair[['date', 'title']]):
            if row['actual'] == pair['previous']:
                new_date = pd.to_datetime(row['date'])
                time_diff = pd.to_datetime(economic_poland[economic_poland['title'] == row['title']]['date']).diff().dt.days.median()
                new_date = new_date - pd.Timedelta(days=time_diff)
                economic_poland.loc[index]['date'] = new_date.strftime('%Y-%m-%d')
            else:
                new_date = pd.to_datetime(pair['date'])
                time_diff = pd.to_datetime(economic_poland[economic_poland['title'] == pair['title']]['date']).diff().dt.days.median()
                new_date = new_date - pd.Timedelta(days=time_diff)
                economic_poland.loc[pair.name]['date'] = new_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(e)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

single positional indexer is out-of-bounds


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [171]:
#Create df with only Poland data
economic_poland['date'] = economic_poland['date'].apply(pd.to_datetime, dayfirst=True)
economic_poland.sort_values(by='date', inplace=True)
economic_poland['date'] = economic_poland['date'].apply(lambda x: x.date().isoformat())
#sorted_dates = economic_poland['date'].sort_values().to_list()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [172]:
#Create new df with continues range of dates and all indicator values in any day

df_pivot = economic_poland.pivot(columns='title', values='actual')

#merge tables to update dates
merged = df_pivot.merge(economic_poland, left_index=True, right_index=True)
merged.drop(['title', 'country', 'indicator', 'comment','actual', 'previous', 'forecast', 'importance'], axis=1, inplace=True)
merged.reset_index(inplace=True)

#create continues dates from oldest to newest 
idx = pd.date_range(merged.date.min(), merged.date.max())
idx = idx.strftime('%Y-%m-%d')

#create new dataframe with full set of date range
new_df = pd.DataFrame(index=idx, columns=merged.columns)
new_df.drop('date', axis=1, inplace=True)

#merge both dataframes table with values in full set of ranges
fullset = pd.merge(new_df, merged, how='left', right_on='date', left_index=True, suffixes=("_x", None))
fullset.dropna(axis=1, how='all', inplace=True)

#fill all nans with values from previous rows(newest)
fullset = fullset.ffill()

#drop duplicates
fullset.drop_duplicates(subset=['date'], inplace=True, keep='last')
fullset.set_index('date', inplace=True)
fullset.drop('index', inplace=True, axis=1)

#fill rest of nan values(oldest data) with oldest 'previous' value from main df
for col in fullset.columns:
    if pd.isna(fullset[col].iloc[0]):
        value = economic_poland.loc[economic_poland['title'] == col].iloc[0]['previous']
        fullset[col] = fullset[col].fillna(value)
        
#check if last row in our dataframe is correct
test_df = pd.DataFrame()
for title in economic_poland['title'].unique():
    test_df[title] = [economic_poland[economic_poland["title"]==title].iloc[-1]["actual"]]
    
test_true = fullset.iloc[-1] == test_df
print(test_true.iloc[0].unique()) #it should only contain "True" values

[ True]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`


In [173]:
error = []
for indicator in economic_poland.title.unique():
    all_rows = economic_poland[economic_poland.title == indicator]
    for index, [idx, row] in enumerate(all_rows.iterrows()):
        try:
            now = row['date']
            until = pd.to_datetime(all_rows.iloc[index+1]['date'])
            until = until - pd.Timedelta(days=1)
            until = until.strftime('%Y-%m-%d')
        except:
            pass
        if now > until:
            temp1 = now
            now = until
            until = temp1
        actual = row['actual']
        if all(actual != fullset.loc[now:until][indicator]):
            error.append([now, until, actual, fullset.loc[now][indicator], indicator])

In [174]:
error

[['2016-03-14', '2016-03-15', -0.9, -0.5, 'Inflation Rate YoY Final'],
 ['2016-03-15', '2016-04-10', -0.5, -0.1, 'Inflation Rate MoM Final'],
 ['2018-03-16', '2018-04-15', 0.8, 1.0, 'Core Inflation Rate YoY'],
 ['2019-03-17', '2019-03-18', 1.0, 0.6, 'Core Inflation Rate YoY'],
 ['2020-03-15', '2020-03-16', 3.6, 3.1, 'Core Inflation Rate YoY'],
 ['2021-03-15', '2021-03-16', 3.9, 3.7, 'Core Inflation Rate YoY'],
 ['2022-03-15', '2022-03-16', 6.1, 5.3, 'Core Inflation Rate YoY'],
 ['2023-03-16', '2023-04-16', 12.0, 11.7, 'Core Inflation Rate YoY']]

In [7]:
#merge both dataframes table with values in full set of ranges
fullset2 = pd.merge(new_df, merged, how='left', right_on='date', left_index=True, suffixes=("_x", None))
fullset2.dropna(axis=1, how='all', inplace=True)
fullset2.set_index('date', inplace=True)

In [44]:
fullset['Inflation Rate YoY Final']['2016-03-01':'2016-03-30']

date
2016-03-01   -0.5
2016-03-02   -0.5
2016-03-03   -0.5
2016-03-04   -0.5
2016-03-05   -0.5
2016-03-06   -0.5
2016-03-07   -0.5
2016-03-08   -0.5
2016-03-09   -0.5
2016-03-10   -0.5
2016-03-11   -0.5
2016-03-12   -0.5
2016-03-13   -0.5
2016-03-14   -0.5
2016-03-15   -0.8
2016-03-16   -0.8
2016-03-17   -0.8
2016-03-18   -0.8
2016-03-19   -0.8
2016-03-20   -0.8
2016-03-21   -0.8
2016-03-22   -0.8
2016-03-23   -0.8
2016-03-24   -0.8
2016-03-25   -0.8
2016-03-26   -0.8
2016-03-27   -0.8
2016-03-28   -0.8
2016-03-29   -0.8
2016-03-30   -0.8
Name: Inflation Rate YoY Final, dtype: float64

In [176]:
mask = economic_poland.duplicated(subset=['date', 'title'], keep=False)
economic_poland.loc[mask]

,date,title,country,indicator,comment,actual,previous,forecast,importance
36099,15-03-2016 13:00:00 +0000,Inflation Rate YoY Final,PL,Inflation Rate,"In Poland, the most important categories in th...",-0.9,-0.5,-0.7,-1
36104,15-03-2016 13:00:00 +0000,Inflation Rate YoY Final,PL,Inflation Rate,"In Poland, the most important categories in th...",-0.8,-0.9,-0.7,-1
36105,15-03-2016 13:00:00 +0000,Inflation Rate MoM Final,PL,Inflation Rate Mom,"In Poland, the most important categories in th...",-0.5,-0.2,-0.4,-1
36106,15-03-2016 13:00:00 +0000,Inflation Rate MoM Final,PL,Inflation Rate Mom,"In Poland, the most important categories in th...",-0.1,-0.5,-0.1,-1
70787,16-03-2018 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",0.8,1.0,NaN,-1
70788,16-03-2018 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",1.0,0.9,1.0,-1
92264,18-03-2019 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",1.0,0.8,NaN,-1
92265,18-03-2019 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",0.8,0.6,0.9,-1
110699,16-03-2020 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",3.6,3.1,NaN,-1
110700,16-03-2020 13:00:00 +0000,Core Inflation Rate YoY,PL,Core Inflation Rate,"In Poland, the core inflation rate tracks chan...",3.1,3.1,3.4,-1


In [ ]:
economic_poland[economic_poland['date'] == '2016-03-15']

In [ ]:
fullset2['Inflation Rate YoY Final'].dropna()[0:20]

In [ ]:
fullset2['Inflation Rate YoY Final']['2016-03-01':'2016-03-30']

In [ ]:
#Forex data
forex_df = pd.read_excel('/kaggle/input/forex-data-gatherer/FOREX_DATA.xlsx')

forex_df['Datetime'] = pd.to_datetime(forex_df['Datetime'], dayfirst=True)
forex_df = dataframe.sort_values(by='Datetime', ascending=False)
forex_df['Datetime'] = forex_df["Datetime"].dt.strftime('%d-%m-%Y %H:%M:%S %z')
forex_df.set_index('Datetime', inplace=True)

In [ ]:
join_df = dataframe.join(calendar_df).drop_duplicates()
join_df = join_df[~join_df.index.duplicated(keep='first')]

In [ ]:
join_df.loc['28-03-2023 07:00:00 +0100']